# IR Lab SoSe 2024: Erweitertes Vergleichendes Retrieval System mit Query Expansion

Dieses Jupyter Notebook implementiert und vergleicht mehrere Retrieval-Ansätze:
1. Nur BM25
2. BM25 + TF-IDF
3. BM25 + Gewichtetes TF-IDF (mit Feldgewichtung)
4. BM25 + Query Expansion

Wir verwenden ein Korpus wissenschaftlicher Arbeiten (Titel + Abstracts) aus den Bereichen Information Retrieval und Natural Language Processing.

### Schritt 1: Bibliotheken importieren und Setup

In [1]:
from tira.third_party_integrations import ensure_pyterrier_is_loaded, persist_and_normalize_run
from tira.rest_api_client import Client
import pyterrier as pt
from pyterrier.pipelines import *
import pandas as pd

ensure_pyterrier_is_loaded()
tira = Client()

# Das Dataset und der Index
pt_dataset = pt.get_dataset('irds:ir-lab-sose-2024/ir-acl-anthology-20240504-training')
index = tira.pt.index('ir-lab-sose-2024/tira-ir-starter/Index (tira-ir-starter-pyterrier)', pt_dataset)

/opt/homebrew/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
PyTerrier 0.10.1 has loaded Terrier 5.7 (built by craigm on 2022-11-10 18:30) and terrier-helper 0.0.7

No etc/terrier.properties, using terrier.default.properties for bootstrap configuration.


### Schritt 2: Retrieval Pipelines definieren

In [2]:
# 1. Nur BM25
bm25 = pt.BatchRetrieve(index, wmodel="BM25")

# 2. BM25 + TF-IDF
tfidf = pt.BatchRetrieve(index, wmodel="TF_IDF")
bm25_tfidf = bm25 >> tfidf

# 3. BM25 + Gewichtetes TF-IDF
fields = ["title", "abstract"]
weights = [2.0, 1.0]  # Titel hat doppeltes Gewicht des Abstracts
weighted_tfidf = pt.BatchRetrieve(index, wmodel="TF_IDF", controls={"c": 1.0}, fields=fields)
for field, weight in zip(fields, weights):
    weighted_tfidf.controls[f"w.{field}"] = weight
bm25_weighted_tfidf = bm25 >> weighted_tfidf

# 4. BM25 + Query Expansion
bo1 = pt.rewrite.Bo1QueryExpansion(index)
bm25_qe = bm25 >> bo1 >> bm25

### Schritt 3: Retrieval durchführen

In [3]:
topics = pt_dataset.get_topics('text')
print('Die ersten drei Topics:')
print(topics.head(3))

print('\nFühre Retrieval für alle Ansätze durch...')
run_bm25 = bm25(topics)
run_bm25_tfidf = bm25_tfidf(topics)
run_bm25_weighted_tfidf = bm25_weighted_tfidf(topics)
run_bm25_qe = bm25_qe(topics)
print('Retrieval abgeschlossen.')

Die ersten drei Topics:
  qid                                     query
0   1  retrieval system improving effectiveness
1   2  machine learning language identification
2   3             social media detect self harm

Führe Retrieval für alle Ansätze durch...


BR(TF_IDF): 100%|██████████| 68/68 [00:01<00:00, 60.81q/s]


Retrieval abgeschlossen.


### Schritt 4: Ergebnisse vergleichen

In [4]:
def compare_top_k(runs, k=10):
    results = {}
    for name, run in runs.items():
        results[name] = run.groupby('qid').apply(lambda x: x.head(k)).reset_index(drop=True)
    
    comparison = pd.DataFrame()
    for name, result in results.items():
        comparison[f'{name}_qid'] = result['qid']
        comparison[f'{name}_docno'] = result['docno']
        comparison[f'{name}_score'] = result['score']
    
    return comparison

runs = {
    'BM25': run_bm25,
    'BM25+TF-IDF': run_bm25_tfidf,
    'BM25+Weighted-TF-IDF': run_bm25_weighted_tfidf,
    'BM25+QE': run_bm25_qe
}

comparison = compare_top_k(runs)
print(f'Vergleich der Top 10 Ergebnisse für die erste Anfrage (qid=1):')
print(comparison[comparison['BM25_qid'] == '1'])

# Berechnen Sie den Overlap zwischen den Top-K Ergebnissen
def calculate_overlap(runs, k=10):
    overlaps = {}
    for name1, run1 in runs.items():
        for name2, run2 in runs.items():
            if name1 < name2:  # Doppellungen bei Vergleichen vermeiden
                overlap = len(set(run1.groupby('qid').head(k)['docno']) & 
                              set(run2.groupby('qid').head(k)['docno']))
                overlaps[f'{name1} vs {name2}'] = overlap
    return overlaps

overlaps = calculate_overlap(runs)
print(f'\nÜberlappung der Top 10 Ergebnisse zwischen den Ansätzen:')
for comparison, overlap in overlaps.items():
    print(f'{comparison}: {overlap} von 10')

# Zeigen Sie die erweiterten Anfragen für BM25+QE
print('\nBeispiele für erweiterte Anfragen (BM25+QE):')
expanded_queries = bm25_qe.transform(topics.head())
for _, row in expanded_queries.iterrows():
    print(f"Original: {row['query']}")
    print(f"Erweitert: {row['query_0']}\n")

Vergleich der Top 10 Ergebnisse für die erste Anfrage (qid=1):
  BM25_qid                          BM25_docno  BM25_score BM25+TF-IDF_qid  \
0        1        2004.cikm_conference-2004.47   15.681777               1   
1        1   1989.ipm_journal-ir0volumeA25A4.2   15.047380               1   
2        1  2005.ipm_journal-ir0volumeA41A5.11   14.144223               1   
3        1                            W05-0704   14.025748               1   
4        1       2016.ntcir_conference-2016.90   13.947994               1   
5        1     1998.sigirconf_conference-98.15   13.901647               1   
6        1       2008.cikm_conference-2008.183   13.808208               1   
7        1                            O01-2005   13.749449               1   
8        1     1998.sigirconf_conference-98.33   13.735541               1   
9        1   2006.ipm_journal-ir0volumeA42A3.2   13.569263               1   

                    BM25+TF-IDF_docno  BM25+TF-IDF_score  \
0        2004.cikm

/var/folders/5k/bwdc2gb51ns2k5373w3_1bhc0000gn/T/ipykernel_53329/4270197507.py:4: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  results[name] = run.groupby('qid').apply(lambda x: x.head(k)).reset_index(drop=True)
/var/folders/5k/bwdc2gb51ns2k5373w3_1bhc0000gn/T/ipykernel_53329/4270197507.py:4: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  results[name] = run.groupby('qid').apply(lambda x: x.head(k)).reset_i

Original: applypipeline:off retriev^1.188537491 system^1.091347342 improv^1.104887992 effect^1.119570106 best^0.062811546 term^0.051147322 signific^0.047031067 demonstr^0.038131943 result^0.022883459 techniqu^0.000000000
Erweitert: retrieval system improving effectiveness

Original: applypipeline:off retriev^1.188537491 system^1.091347342 improv^1.104887992 effect^1.119570106 best^0.062811546 term^0.051147322 signific^0.047031067 demonstr^0.038131943 result^0.022883459 techniqu^0.000000000
Erweitert: retrieval system improving effectiveness

Original: applypipeline:off retriev^1.188537491 system^1.091347342 improv^1.104887992 effect^1.119570106 best^0.062811546 term^0.051147322 signific^0.047031067 demonstr^0.038131943 result^0.022883459 techniqu^0.000000000
Erweitert: retrieval system improving effectiveness

Original: applypipeline:off retriev^1.188537491 system^1.091347342 improv^1.104887992 effect^1.119570106 best^0.062811546 term^0.051147322 signific^0.047031067 demonstr^0.0381319

### Schritt 5: Run-Dateien persistieren

In [5]:
persist_and_normalize_run(run_bm25, system_name='bm25-baseline', default_output='../runs')
persist_and_normalize_run(run_bm25_tfidf, system_name='bm25-tfidf-combined', default_output='../runs')
persist_and_normalize_run(run_bm25_weighted_tfidf, system_name='bm25-weighted-tfidf-combined', default_output='../runs')
persist_and_normalize_run(run_bm25_qe, system_name='bm25-query-expansion', default_output='../runs')

The run file is normalized outside the TIRA sandbox, I will store it at "../runs".
Done. run file is stored under "../runs/run.txt".
The run file is normalized outside the TIRA sandbox, I will store it at "../runs".
Done. run file is stored under "../runs/run.txt".
The run file is normalized outside the TIRA sandbox, I will store it at "../runs".
Done. run file is stored under "../runs/run.txt".
The run file is normalized outside the TIRA sandbox, I will store it at "../runs".
Done. run file is stored under "../runs/run.txt".


### Schritt 6: Analyse und Interpretation

1. Vergleichen Sie die Top-10-Ergebnisse für verschiedene Anfragen. Wie unterscheiden sich die Ergebnisse zwischen den vier Ansätzen?

2. Analysieren Sie die Überlappung der Ergebnisse. Wie ähnlich sind die Ergebnisse der verschiedenen Methoden?

3. Betrachten Sie die erweiterten Anfragen des Query Expansion-Ansatzes. Sind die hinzugefügten Terme relevant und hilfreich?

4. Untersuchen Sie Fälle, in denen Query Expansion zu deutlich anderen Ergebnissen führt als die anderen Methoden. Sind diese Unterschiede auf die zusätzlichen Terme zurückzuführen?

5. Vergleichen Sie die Ergebnisse des gewichteten TF-IDF-Ansatzes mit denen der Query Expansion. Welcher Ansatz scheint effektiver zu sein?

6. Betrachten Sie die Scores der verschiedenen Methoden. Wie unterscheiden sie sich in ihrer Verteilung und Größenordnung?

7. Führen Sie eine manuelle Stichprobenprüfung der Ergebnisse für einige ausgewählte Anfragen durch, um die Qualität der verschiedenen Ansätze zu beurteilen.

8. Überlegen Sie, wie Sie die Ansätze weiter verbessern könnten:
   - Experimentieren mit verschiedenen Query Expansion-Techniken
   - Kombination von Query Expansion mit gewichtetem TF-IDF
   - Anpassung der Parameter für Query Expansion (z.B. Anzahl der hinzugefügten Terme)
   - Einbeziehung von domänenspezifischem Wissen in die Query Expansion

Beachten Sie, dass die Effektivität jeder Methode von der spezifischen Aufgabe und dem Datensatz abhängt. Eine gründliche Evaluation ist entscheidend, um zu verstehen, welcher Ansatz für Ihre spezifischen Bedürfnisse am besten geeignet ist.